In [2]:
import os
import pandas as pd
import patsy
import numpy as np

In [3]:
if not os.getcwd().endswith('assignment3'):
    os.chdir(os.path.join('assignment3'))
# read the file
parole = pd.read_csv('parole.csv')

# rename problematic coulmns
parole = parole.rename(columns={'time.served': 'timeServed', 
                                'max.sentence': 'maxSentence', 
                                'multiple.offenses': 'multipleOffenses'})

### Problem 1.1 - number of observations

In [4]:
len(parole.index)

675

### Problem 1.1 - number of voilations

In [5]:
(parole['violator'] == 1).sum()

78

### Problem 2.1 - convert to categorical

I couldn't find any python parallel to R's .asfactor
Instead, we'll use the patsy.dmatrices

### Problem 3.1 - Splitting into a Training and Testing Set

In [7]:
# scikit-learn used the numpy random number generator (?) 
np.random.seed(7)
from sklearn.model_selection import train_test_split
parole_train, parole_test = train_test_split(parole, test_size=0.3)

In [8]:
parole_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472 entries, 82 to 175
Data columns (total 9 columns):
male                472 non-null int64
race                472 non-null int64
age                 472 non-null float64
state               472 non-null int64
timeServed          472 non-null float64
maxSentence         472 non-null int64
multipleOffenses    472 non-null int64
crime               472 non-null int64
violator            472 non-null int64
dtypes: float64(2), int64(7)
memory usage: 36.9 KB


### Problem 4.1 - Building a Logistic Regression Model

In [10]:
from statsmodels.discrete.discrete_model import Logit

In [189]:
# we will use pats to generate the matrices,
# the C() operator means - treat as categorical 

y, X = patsy.dmatrices('violator ~ male + race + age + C(state) + '
                       'timeServed + maxSentence + multipleOffenses + '
                       'C(crime)', parole_train, return_type='dataframe')
mod1 = Logit(y, X).fit()
mod1.summary2()

Optimization terminated successfully.
         Current function value: 0.246405
         Iterations 8


<table class="simpletable">
<tr>
        <td>Model:</td>              <td>Logit</td>      <td>Pseudo R-squared:</td>    <td>0.315</td>  
</tr>
<tr>
  <td>Dependent Variable:</td>     <td>violator</td>           <td>AIC:</td>         <td>258.6063</td> 
</tr>
<tr>
         <td>Date:</td>        <td>2017-03-26 00:40</td>       <td>BIC:</td>         <td>312.6470</td> 
</tr>
<tr>
   <td>No. Observations:</td>         <td>472</td>        <td>Log-Likelihood:</td>    <td>-116.30</td> 
</tr>
<tr>
       <td>Df Model:</td>             <td>12</td>            <td>LL-Null:</td>        <td>-169.89</td> 
</tr>
<tr>
     <td>Df Residuals:</td>           <td>459</td>         <td>LLR p-value:</td>    <td>2.1568e-17</td>
</tr>
<tr>
      <td>Converged:</td>           <td>1.0000</td>           <td>Scale:</td>         <td>1.0000</td>  
</tr>
<tr>
    <td>No. Iterations:</td>        <td>8.0000</td>              <td></td>               <td></td>     
</tr>
</table>
<table class="simpletable">
<tr>
          <td></td>          <th>Coef.</th>  <th>Std.Err.</th>    <th>z</th>     <th>P>|z|</th> <th>[0.025</th>  <th>0.975]</th> 
</tr>
<tr>
  <th>Intercept</th>        <td>-5.1002</td>  <td>1.3713</td>  <td>-3.7191</td> <td>0.0002</td> <td>-7.7879</td> <td>-2.4124</td>
</tr>
<tr>
  <th>C(state)[T.2]</th>    <td>0.2289</td>   <td>0.5125</td>  <td>0.4465</td>  <td>0.6552</td> <td>-0.7757</td> <td>1.2334</td> 
</tr>
<tr>
  <th>C(state)[T.3]</th>    <td>1.2172</td>   <td>0.5506</td>  <td>2.2106</td>  <td>0.0271</td> <td>0.1380</td>  <td>2.2965</td> 
</tr>
<tr>
  <th>C(state)[T.4]</th>    <td>-3.2544</td>  <td>0.6330</td>  <td>-5.1414</td> <td>0.0000</td> <td>-4.4950</td> <td>-2.0138</td>
</tr>
<tr>
  <th>C(crime)[T.2]</th>    <td>0.4879</td>   <td>0.5332</td>  <td>0.9150</td>  <td>0.3602</td> <td>-0.5572</td> <td>1.5330</td> 
</tr>
<tr>
  <th>C(crime)[T.3]</th>    <td>-0.1386</td>  <td>0.4321</td>  <td>-0.3207</td> <td>0.7485</td> <td>-0.9855</td> <td>0.7084</td> 
</tr>
<tr>
  <th>C(crime)[T.4]</th>    <td>-0.6568</td>  <td>0.6431</td>  <td>-1.0212</td> <td>0.3072</td> <td>-1.9173</td> <td>0.6037</td> 
</tr>
<tr>
  <th>male</th>             <td>0.7200</td>   <td>0.4980</td>  <td>1.4457</td>  <td>0.1483</td> <td>-0.2561</td> <td>1.6961</td> 
</tr>
<tr>
  <th>race</th>             <td>0.4665</td>   <td>0.3949</td>  <td>1.1813</td>  <td>0.2375</td> <td>-0.3075</td> <td>1.2405</td> 
</tr>
<tr>
  <th>age</th>              <td>0.0220</td>   <td>0.0172</td>  <td>1.2788</td>  <td>0.2010</td> <td>-0.0117</td> <td>0.0558</td> 
</tr>
<tr>
  <th>timeServed</th>       <td>-0.0935</td>  <td>0.1257</td>  <td>-0.7442</td> <td>0.4568</td> <td>-0.3398</td> <td>0.1528</td> 
</tr>
<tr>
  <th>maxSentence</th>      <td>0.0774</td>   <td>0.0573</td>  <td>1.3497</td>  <td>0.1771</td> <td>-0.0350</td> <td>0.1898</td> 
</tr>
<tr>
  <th>multipleOffenses</th> <td>1.9858</td>   <td>0.4087</td>  <td>4.8591</td>  <td>0.0000</td> <td>1.1848</td>  <td>2.7868</td> 
</tr>
</table>

The important variables (probability less then 0.05) are "race", "state4" and "multipleOffenses"

### problem 4.2
What can we say based on the coefficient of the multiple.offenses variable? answer: Our model predicts that a parolee who committed multiple offenses has X times higher odds of being a violator than a parolee who did not commit multiple offenses but is otherwise identical

About the answer:
$$yes/no = exp(\beta_0+\beta_1x)$$ is the odds ratio of a parolee to violate the parole (yes) vs not violating the parole (no). If the coefficient for multipleOffenses is $C_{mOf}$ than he has chances greater by $exp(C_{mOf})$ of violating the parole.

In [190]:
mOf = mod1.params['multipleOffenses']
np.exp(mOf)

7.284746722326692

### Problem 4.3 - Building a Logistic Regression Model
Consider a parolee who is male, of white race, aged 50 years at prison release, from the state of Maryland, served 3 months, had a maximum sentence of 12 months, did not commit multiple offenses, and committed a larceny.

Answer: this is an easy question. As a reminder, the probability of violating the parole is: $$ P(Y=1) = \frac{1}{1+exp(-1*( ... ))}$$

### Problem 5.1 - Evaluating the Model on the Testing Set
Predict probabilities for parolees in the testing set. What is the maximal?

In [191]:
y_test, X_test = patsy.dmatrices('violator ~ male + race + age + C(state) + '
                       'timeServed + maxSentence + multipleOffenses + '
                       'C(crime)', parole_test, return_type='dataframe')
precitions_mod1 = mod1.predict(exog=X_test)
precitions_mod1.max()

0.87611561665337234

### Problem 5.2 - Evaluating the Model on the Testing Set
evaluate the model's predictions on the test set using a threshold of 0.5. What is the model's sensitivity? specificity? accuracy?

Let's generate the confusion table:

In [192]:
from sklearn.metrics import confusion_matrix

thresh = 0.5
con_matx = confusion_matrix(parole_test['violator'].values, precitions_mod1 > thresh)
con_matx

array([[169,  11],
       [ 15,   8]])

In [193]:
TP = con_matx[0, 1]
TN = con_matx[0, 0]
FP = con_matx[1, 1]
FN = con_matx[1, 0]

In [194]:
# sensitivity
(TP) / (TP + FN)

0.42307692307692307

In [195]:
# specificity
TN / (TN + FP)

0.95480225988700562

In [196]:
# accuracy
(TP + TN) / len(precitions_mod1.index)

0.88669950738916259

### Problem 5.3 - What is the accuracy of a simple model that predicts that every parolee is a non-violator?

In [197]:
# In that case:
# BTW we choose > 1 because it is impossible for probability to be greater than 1 
con_matx = confusion_matrix(parole_test['violator'].values, precitions_mod1 > 1)

TP = con_matx[0, 1]
TN = con_matx[0, 0]
FP = con_matx[1, 1]
FN = con_matx[1, 0]

# the accuracy:
(TP + TN) / len(precitions_mod1.index)

0.88669950738916259

Comment: this probably could have been done more efficiently

### Problem 5.6 - what is the AUC value for the model?

In [198]:
from sklearn.metrics import roc_auc_score as auc
auc(y_true=parole_test['violator'].values, y_score=precitions_mod1)

0.64335748792270531